In [76]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubun

In [1]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-09 17:00:36--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2022-01-09 17:00:36 (11.2 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Animal_Shelter_Project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [77]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://c-geisel-bucket.s3.amazonaws.com/Breed_data.csv"
spark.sparkContext.addFile(url)
dog_breed_df = spark.read.csv(SparkFiles.get("Breed_data.csv"), sep=",", header=True, inferSchema=True)
dog_breed_df.show()

+--------------------+------+---------------+
|               Breed|  Size|life_expectancy|
+--------------------+------+---------------+
|        Pit Bull Mix| large|             11|
|Chihuahua Shortha...| small|             15|
|Labrador Retrieve...|large |             13|
| German Shepherd Mix| large|             10|
|Australian Cattle...|medium|             13|
|       Dachshund Mix| small|             14|
|           Boxer Mix| large|              9|
|Miniature Poodle Mix| small|             14|
|   Border Collie Mix|medium|             12|
|       Catahoula Mix| large|             12|
|     Rat Terrier Mix| small|             16|
|Australian Shephe...|medium|             13|
|Yorkshire Terrier...| small|             14|
|  Siberian Husky Mix|medium|             13|
|Jack Russell Terr...| small|             15|
+--------------------+------+---------------+



In [27]:
#read in the uncleaned shelter info 
from pyspark import SparkFiles
url = "https://c-geisel-bucket.s3.amazonaws.com/aac_shelter_outcomes.csv"
spark.sparkContext.addFile(url)
uncleaned_shelter_df = spark.read.csv(SparkFiles.get("aac_shelter_outcomes.csv"), sep=",", header=True, inferSchema=True)
uncleaned_shelter_df.show()

+----------------+---------+-----------+--------------------+-------------------+-------------------+-------------------+-------------------+-----------+---------------+---------------+----------------+
|age_upon_outcome|animal_id|animal_type|               breed|              color|      date_of_birth|           datetime|          monthyear|       name|outcome_subtype|   outcome_type|sex_upon_outcome|
+----------------+---------+-----------+--------------------+-------------------+-------------------+-------------------+-------------------+-----------+---------------+---------------+----------------+
|         2 weeks|  A684346|        Cat|Domestic Shorthai...|       Orange Tabby|2014-07-07 00:00:00|2014-07-22 16:04:00|2014-07-22 16:04:00|       null|        Partner|       Transfer|     Intact Male|
|          1 year|  A666430|        Dog|          Beagle Mix|        White/Brown|2012-11-06 00:00:00|2013-11-07 11:47:00|2013-11-07 11:47:00|       Lucy|        Partner|       Transfer|   

###Turn Spark DataFrames into Pandas Dataframes

In [28]:
#turn this into a pandas dataframe 
import pandas as pd

In [29]:
df = uncleaned_shelter_df.toPandas()


In [30]:
df.head()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07,2014-07-22 16:04:00,2014-07-22 16:04:00,None,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06,2013-11-07 11:47:00,2013-11-07 11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31,2014-06-03 14:20:00,2014-06-03 14:20:00,*Johnny,None,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02,2014-06-15 15:50:00,2014-06-15 15:50:00,Monday,Partner,Transfer,Neutered Male
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07,2014-07-07 14:04:00,2014-07-07 14:04:00,None,Rabies Risk,Euthanasia,Unknown


In [78]:
breed_df = dog_breed_df.toPandas()

In [79]:
breed_df.head(5)

,Breed,Size,life_expectancy
0,Pit Bull Mix,large,11
1,Chihuahua Shorthair Mix,small,15
2,Labrador Retriever Mix,large,13
3,German Shepherd Mix,large,10
4,Australian Cattle Dog Mix,medium,13


In [80]:
breed_df = breed_df.rename(columns={'Breed': 'breed'})

In [81]:
breed_df.head(5)

,breed,Size,life_expectancy
0,Pit Bull Mix,large,11
1,Chihuahua Shorthair Mix,small,15
2,Labrador Retriever Mix,large,13
3,German Shepherd Mix,large,10
4,Australian Cattle Dog Mix,medium,13


###Clean the shelter dataframe

In [31]:
#Filter so that only dog data is shown
dfdogs = df[df["animal_type"] == "Dog"]

In [32]:
dfdogs

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06,2013-11-07 11:47:00,2013-11-07 11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31,2014-06-03 14:20:00,2014-06-03 14:20:00,*Johnny,None,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02,2014-06-15 15:50:00,2014-06-15 15:50:00,Monday,Partner,Transfer,Neutered Male
5,4 months,A664462,Dog,Leonberger Mix,Brown/White,2013-06-03,2013-10-07 13:06:00,2013-10-07 13:06:00,*Edgar,Partner,Transfer,Intact Male
7,3 years,A692618,Dog,Chihuahua Shorthair Mix,Brown,2011-11-23,2014-12-08 15:55:00,2014-12-08 15:55:00,*Ella,Partner,Transfer,Spayed Female
...,...,...,...,...,...,...,...,...,...,...,...,...
78250,1 month,A764895,Dog,Golden Retriever/Labrador Retriever,Brown/White,2017-12-04,2018-02-01 18:40:00,2018-02-01 18:40:00,None,Foster,Adoption,Neutered Male
78251,1 month,A764894,Dog,Golden Retriever/Labrador Retriever,Brown/White,2017-12-04,2018-02-01 18:26:00,2018-02-01 18:26:00,None,Foster,Adoption,Spayed Female
78252,3 years,A764468,Dog,Mastiff Mix,Blue/White,2014-12-30,2018-02-01 18:06:00,2018-02-01 18:06:00,Max,None,Adoption,Neutered Male
78254,2 months,A765858,Dog,Standard Schnauzer,Red,2017-11-13,2018-02-01 18:32:00,2018-02-01 18:32:00,None,None,Adoption,Spayed Female


In [33]:
#drop unnecessary columns
dfdogs_cleaned = dfdogs.drop(columns = ["age_upon_outcome", "animal_type", "color", "name", "outcome_subtype", "monthyear"])

In [34]:
dfdogs_cleaned.head(15)

,animal_id,breed,date_of_birth,datetime,outcome_type,sex_upon_outcome
1,A666430,Beagle Mix,2012-11-06,2013-11-07 11:47:00,Transfer,Spayed Female
2,A675708,Pit Bull,2013-03-31,2014-06-03 14:20:00,Adoption,Neutered Male
3,A680386,Miniature Schnauzer Mix,2005-06-02,2014-06-15 15:50:00,Transfer,Neutered Male
5,A664462,Leonberger Mix,2013-06-03,2013-10-07 13:06:00,Transfer,Intact Male
7,A692618,Chihuahua Shorthair Mix,2011-11-23,2014-12-08 15:55:00,Transfer,Spayed Female
11,A673652,Papillon/Border Collie,2012-02-28,2014-03-28 14:39:00,Transfer,Neutered Male
12,A677679,Chihuahua Shorthair/Pomeranian,2014-03-07,2014-05-26 19:10:00,Adoption,Neutered Male
13,A640655,Miniature Schnauzer/Miniature Poodle,2009-04-27,2014-04-25 11:17:00,Return to Owner,Spayed Female
14,A690350,Labrador Retriever Mix,2006-10-18,2014-10-26 18:20:00,Return to Owner,Neutered Male
15,A680396,Rat Terrier Mix,2012-06-02,2014-06-15 15:11:00,Transfer,Neutered Male


In [35]:
#change the datatype of the date columns
dfdogs_cleaned["date_of_birth"] = pd.to_datetime(dfdogs_cleaned["date_of_birth"])
dfdogs_cleaned["datetime"] = pd.to_datetime(dfdogs_cleaned["datetime"])

In [36]:
dfdogs_cleaned.head(15)

,animal_id,breed,date_of_birth,datetime,outcome_type,sex_upon_outcome
1,A666430,Beagle Mix,2012-11-06,2013-11-07 11:47:00,Transfer,Spayed Female
2,A675708,Pit Bull,2013-03-31,2014-06-03 14:20:00,Adoption,Neutered Male
3,A680386,Miniature Schnauzer Mix,2005-06-02,2014-06-15 15:50:00,Transfer,Neutered Male
5,A664462,Leonberger Mix,2013-06-03,2013-10-07 13:06:00,Transfer,Intact Male
7,A692618,Chihuahua Shorthair Mix,2011-11-23,2014-12-08 15:55:00,Transfer,Spayed Female
11,A673652,Papillon/Border Collie,2012-02-28,2014-03-28 14:39:00,Transfer,Neutered Male
12,A677679,Chihuahua Shorthair/Pomeranian,2014-03-07,2014-05-26 19:10:00,Adoption,Neutered Male
13,A640655,Miniature Schnauzer/Miniature Poodle,2009-04-27,2014-04-25 11:17:00,Return to Owner,Spayed Female
14,A690350,Labrador Retriever Mix,2006-10-18,2014-10-26 18:20:00,Return to Owner,Neutered Male
15,A680396,Rat Terrier Mix,2012-06-02,2014-06-15 15:11:00,Transfer,Neutered Male


In [37]:
#Filter the dataframe to show only the top 15 dog breeds
top_breeds = ['Pit Bull Mix', 'Chihuahua Shorthair Mix', 'Labrador Retriever Mix', 'German Shepherd Mix', 'Australian Cattle Dog Mix',
              'Dachshund Mix', 'Boxer Mix', 'Miniature Poodle Mix', 'Border Collie Mix', 'Catahoula Mix', 'Rat Terrier Mix', 
              'Australian Shepherd Mix', 'Yorkshire Terrier Mix', 'Siberian Husky Mix', 'Jack Russell Terrier Mix']
dfdogs_cleaned = dfdogs_cleaned[dfdogs_cleaned.breed.isin(top_breeds)]

In [39]:
dfdogs_cleaned.head(10)

,animal_id,breed,date_of_birth,datetime,outcome_type,sex_upon_outcome
7,A692618,Chihuahua Shorthair Mix,2011-11-23,2014-12-08 15:55:00,Transfer,Spayed Female
14,A690350,Labrador Retriever Mix,2006-10-18,2014-10-26 18:20:00,Return to Owner,Neutered Male
15,A680396,Rat Terrier Mix,2012-06-02,2014-06-15 15:11:00,Transfer,Neutered Male
16,A674298,Pit Bull Mix,2013-03-11,2014-04-16 12:51:00,Transfer,Neutered Male
27,A667311,Labrador Retriever Mix,2013-09-01,2013-11-19 18:30:00,Adoption,Spayed Female
28,A690699,Chihuahua Shorthair Mix,2014-06-17,2014-11-04 18:03:00,Transfer,Intact Female
32,A669918,Labrador Retriever Mix,2013-09-30,2014-01-03 17:47:00,Adoption,Spayed Female
34,A678098,Labrador Retriever Mix,2002-05-08,2014-07-09 11:54:00,Adoption,Neutered Male
36,A677038,Chihuahua Shorthair Mix,2012-04-22,2014-04-26 13:00:00,Adoption,Spayed Female
37,A665344,Chihuahua Shorthair Mix,2009-10-16,2013-11-02 17:47:00,Adoption,Spayed Female


In [40]:
#reset the dataframe index
dfdogs_cleaned.reset_index(drop=True)

,animal_id,breed,date_of_birth,datetime,outcome_type,sex_upon_outcome
0,A692618,Chihuahua Shorthair Mix,2011-11-23,2014-12-08 15:55:00,Transfer,Spayed Female
1,A690350,Labrador Retriever Mix,2006-10-18,2014-10-26 18:20:00,Return to Owner,Neutered Male
2,A680396,Rat Terrier Mix,2012-06-02,2014-06-15 15:11:00,Transfer,Neutered Male
3,A674298,Pit Bull Mix,2013-03-11,2014-04-16 12:51:00,Transfer,Neutered Male
4,A667311,Labrador Retriever Mix,2013-09-01,2013-11-19 18:30:00,Adoption,Spayed Female
...,...,...,...,...,...,...
23845,A738395,Chihuahua Shorthair Mix,2015-11-14,2018-02-01 18:23:00,Adoption,Neutered Male
23846,A761266,Border Collie Mix,2015-11-01,2018-02-01 18:32:00,Return to Owner,Spayed Female
23847,A725872,German Shepherd Mix,2016-02-03,2018-02-01 18:20:00,Return to Owner,Intact Female
23848,A761265,Border Collie Mix,2015-11-01,2018-02-01 18:31:00,Return to Owner,Neutered Male


In [43]:
#turn the cleaned dataframe back into a spark dataframe so that it can be written into the RDS instance
dfdogs_cleaned_spark = spark.createDataFrame(dfdogs_cleaned)

In [44]:
dfdogs_cleaned_spark.show()

+---------+--------------------+-------------------+-------------------+---------------+----------------+
|animal_id|               breed|      date_of_birth|           datetime|   outcome_type|sex_upon_outcome|
+---------+--------------------+-------------------+-------------------+---------------+----------------+
|  A692618|Chihuahua Shortha...|2011-11-23 00:00:00|2014-12-08 15:55:00|       Transfer|   Spayed Female|
|  A690350|Labrador Retrieve...|2006-10-18 00:00:00|2014-10-26 18:20:00|Return to Owner|   Neutered Male|
|  A680396|     Rat Terrier Mix|2012-06-02 00:00:00|2014-06-15 15:11:00|       Transfer|   Neutered Male|
|  A674298|        Pit Bull Mix|2013-03-11 00:00:00|2014-04-16 12:51:00|       Transfer|   Neutered Male|
|  A667311|Labrador Retrieve...|2013-09-01 00:00:00|2013-11-19 18:30:00|       Adoption|   Spayed Female|
|  A690699|Chihuahua Shortha...|2014-06-17 00:00:00|2014-11-04 18:03:00|       Transfer|   Intact Female|
|  A669918|Labrador Retrieve...|2013-09-30 00:

###Write the Cleaned Dataframes into tables in RDS

In [82]:
# Store environmental variable
from getpass import getpass
password = getpass('Whitedog24')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cbl71rrsnrzm.us-east-1.rds.amazonaws.com:5432/shelter_data"
config = {"user":"postgres",
          "password": "Whitedog24",
          "driver":"org.postgresql.Driver"}

Whitedog24··········


In [83]:
# Write breed_df to table in RDS (The csv that lexie made)
#must use a spark df
dog_breed_df.write.jdbc(url=jdbc_url, table='breed_info', mode=mode, properties=config)

In [45]:
# Write review_id_df to table in RDS (The data that is cleaned from kaggle)
#must use a spark df
dfdogs_cleaned_spark.write.jdbc(url=jdbc_url, table='shelter_info', mode=mode, properties=config)

###Join the Two Tables on the Dog Breed Column

In [87]:
#Concatenate the two dataframes
shelter_df = dfdogs_cleaned.merge(breed_df, how='inner', on='breed', right_index=False)

In [88]:
shelter_df

,animal_id,breed,date_of_birth,datetime,outcome_type,sex_upon_outcome,Size,life_expectancy
0,A692618,Chihuahua Shorthair Mix,2011-11-23,2014-12-08 15:55:00,Transfer,Spayed Female,small,15
1,A690699,Chihuahua Shorthair Mix,2014-06-17,2014-11-04 18:03:00,Transfer,Intact Female,small,15
2,A677038,Chihuahua Shorthair Mix,2012-04-22,2014-04-26 13:00:00,Adoption,Spayed Female,small,15
3,A665344,Chihuahua Shorthair Mix,2009-10-16,2013-11-02 17:47:00,Adoption,Spayed Female,small,15
4,A677479,Chihuahua Shorthair Mix,2013-04-25,2014-05-01 17:05:00,Adoption,Spayed Female,small,15
...,...,...,...,...,...,...,...,...
23845,A765233,Catahoula Mix,2017-06-15,2018-01-17 16:50:00,Transfer,Intact Female,large,12
23846,A648633,Catahoula Mix,2012-12-17,2018-01-26 15:29:00,Return to Owner,Spayed Female,large,12
23847,A689738,Catahoula Mix,2012-10-08,2018-01-27 15:12:00,Adoption,Spayed Female,large,12
23848,A764311,Catahoula Mix,2016-12-27,2017-12-31 15:54:00,Adoption,Neutered Male,large,12


###Create a Machine Learning Model